In [21]:
# Install microdf
!pip install git+https://github.com/PSLmodels/microdf.git
# update plotly
!pip install plotly --upgrade

# Install UBI Center
!pip install git+http://github.com/ubicenter/ubicenter.py

  Cloning https://github.com/PSLmodels/microdf.git to /tmp/pip-req-build-vlrkdi54
  Running command git clone -q https://github.com/PSLmodels/microdf.git /tmp/pip-req-build-vlrkdi54
  Cloning http://github.com/ubicenter/ubicenter.py to /tmp/pip-req-build-cquxa5fs
  Running command git clone -q http://github.com/ubicenter/ubicenter.py /tmp/pip-req-build-cquxa5fs


In [22]:
# Add Colors
BLUE = '#1976D2'
DARK_BLUE = '#1565C0'
LIGHT_BLUE = '#90CAF9'
GRAY = '#BDBDBD'

In [23]:
# Import libraries
import pandas as pd
import numpy as np
import microdf as mdf
import plotly.express as px
from ubicenter import format_fig
import plotly.graph_objects as go

In [24]:
# Import data
person = pd.read_csv('https://github.com/MaxGhenis/datarepo/raw/master/pppub20.csv.gz',
                         usecols=['MARSUPWT', 'SPM_ID','SPM_POVTHRESHOLD',
                                  'SPM_RESOURCES', 'A_AGE', 'TAX_ID', 
                                   'AGI',  'SPM_WEIGHT', 'DEP_STAT', 'FILESTAT'
                                  ])

In [25]:
# Setup
person.columns = person.columns.str.lower()
person['weight'] = person.marsupwt/100
person.spm_weight = person.spm_weight / 100
person.agi.replace({99999999: 0},inplace=True)

In [26]:
# Define child and adult
person['child'] = person.a_age < 18
person['adult'] = person.a_age >= 18
person["child_dependent"] = person.child & (person.dep_stat > 0)

In [27]:
# Calculate population
population = person.weight.sum()
adult_population = (person.weight * person.adult).sum()
child_population = (person.weight * person.child).sum()

In [28]:
# filestat values:
# 1 = joint, both<65
# 2 = joint, one ><65 & one 65+
# 3 = joint, both 65+
# 4 = head of household
# 5 = single
# 6 = non-filer
# Reorder so all family tax filer status is the same
# joint = 3, HofH = 2, single = 1, non_filer = 0
person['filer_status'] = np.where(person.filestat <=3, "3. Joint",
                        np.where(person.filestat == 4, "2. HoH",
                        np.where(person.filestat == 5, "1. Single",
                                 "0. Non-filer")))
# Override non-filer statuses to be single if they are not dependents.
person.loc[(person.filestat == 6) & (person.dep_stat == 0), "filer_status"] = "1. Single"
# There should only be one non-dependent person in each non-filing tax unit.
assert person[(person.filestat == 6) & (person.dep_stat == 0)].groupby("tax_id").size().max() == 1
tax_unit_filer_status = person.groupby("tax_id")[['filer_status']].max()

In [29]:
# Calculate total children, adults, and AGI per tax unit excluding adult dependents
tax_unit = person.groupby(['spm_id', 'spm_weight','tax_id'])[['child_dependent', 'adult', 'agi']].sum().reset_index() #, 'weight']].sum()
# tax_unit = tax_unit.reset_index()
# tax_unit.columns = ['spm_id', 'spm_weight','tax_id', 'total_children', 'total_adults',  'total_agi', 'tax_unit_weight']
tax_unit = tax_unit.merge(tax_unit_filer_status, on='tax_id')

In [30]:
# Set numbers from proposal
AMOUNT_PER_ADULT = 14_400
AMOUNT_PER_CHILD = 7_200
PHASE_START_JOINT = 150_000
PHASE_START_HEAD = 112_500
PHASE_START_SINGLE = 75_000

In [31]:
# Calculate tax units income floor
tax_unit['income_floor'] = AMOUNT_PER_ADULT * np.where(tax_unit.filer_status == "3. Joint", 2, np.where(tax_unit.filer_status == "0. Non-filer", 0, 1)) + AMOUNT_PER_CHILD * tax_unit.child_dependent
# NB: A few people have dep_stat pointers, but they're the only people in their tax units. These are adult dependents and we give them $0.

In [32]:
# Calculate households phase start
tax_unit['phase_start']  = np.where(tax_unit.filer_status == "3. Joint", PHASE_START_JOINT,
                          np.where(tax_unit.filer_status == "2. HoH", PHASE_START_HEAD, PHASE_START_SINGLE))

In [33]:
# Calculate income above phase out
tax_unit['income_above_phase_start'] = np.maximum(tax_unit.agi - tax_unit.phase_start, 0)

In [34]:
PHASE_RATE = 0.05
# Calculate the amount of GI phase out
tax_unit['phased_out'] = PHASE_RATE * tax_unit.income_above_phase_start

In [35]:
# Calculate total transfer of tax unit
tax_unit['transfer'] = np.maximum(tax_unit.income_floor - tax_unit.phased_out, 0)

In [36]:
tax_unit.sort_values("transfer")

,spm_id,spm_weight,tax_id,child_dependent,adult,agi,filer_status,income_floor,phase_start,income_above_phase_start,phased_out,transfer
69826,80014001,1440.65,8001401,2,2,1118051,3. Joint,43200,150000,968051,48402.55,0.0
44062,50856001,2964.78,5085601,0,1,384510,1. Single,14400,75000,309510,15475.50,0.0
77233,86607001,1778.62,8660701,2,2,2332074,3. Joint,43200,150000,2182074,109103.70,0.0
12270,14279001,2481.44,1427901,1,4,1996251,3. Joint,36000,150000,1846251,92312.55,0.0
77703,87076001,2307.48,8707601,0,1,382000,1. Single,14400,75000,307000,15350.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
61613,71224001,204.51,7122401,9,2,15180,3. Joint,93600,150000,0,0.00,93600.0
12926,15037001,2271.56,1503701,9,2,51456,3. Joint,93600,150000,0,0.00,93600.0
31727,36687001,2276.63,3668701,9,3,121595,3. Joint,93600,150000,0,0.00,93600.0
15429,17944001,2467.55,1794401,10,2,102505,3. Joint,100800,150000,0,0.00,100800.0


In [37]:
# Calculate the total transfer to each SPM Unit and the total cost
spmu = tax_unit.groupby(['spm_id', 'spm_weight'])['transfer'].sum().reset_index()
cost = (spmu.spm_weight * spmu.transfer).sum()
cost / 1e12

3.7530212846027404

In [38]:
# Merge transfer onto person to calculate poverty
person = person.merge(spmu[["spm_id", "transfer"]], on=['spm_id'])

In [41]:
# Calculate % of people that live in SPM Unit that gets a check
person['got_transfer'] = person.transfer > 0
mdf.weighted_mean(person, "got_transfer", "weight")

0.995622446891318

In [42]:
# Calculate total cost with no phase out
ubi_cost = AMOUNT_PER_ADULT * adult_population + AMOUNT_PER_CHILD * child_population 

In [43]:
# Calculate savings from means test and dependent limitation in billions.
(ubi_cost - cost) / 1e9

404.15282391725975

In [65]:
# Calculate new resources
person['spm_resources_new'] = person.transfer + person.spm_resources

In [45]:
# Calculate original poverty rate
person['original_poor'] = person.spm_resources < person.spm_povthreshold

og_total_poor = (person.original_poor * person.weight).sum()
og_pov_rate = (og_total_poor / population * 100).round(1)

In [46]:
# Calculate original adult poverty rate
og_adult_poor = (person.original_poor * person.adult * person.weight).sum()
og_adult_pov_rate = (og_adult_poor / adult_population * 100).round(1)

/usr/local/lib/python3.7/dist-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [47]:
# Calculate original child poverty rate
og_child_poor = (person.original_poor * person.child * person.weight).sum()
og_child_pov_rate = (og_child_poor / child_population * 100).round(1)

/usr/local/lib/python3.7/dist-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [48]:
# Calculate new poverty rate
person['new_poor'] = person.new_resources < person.spm_povthreshold

new_total_poor = (person.new_poor * person.weight).sum()
new_pov_rate = (new_total_poor / population * 100).round(1)

In [49]:
# Calculate original adult poverty rate
new_adult_poor = (person.new_poor * person.adult * person.weight).sum()
new_adult_pov_rate = (new_adult_poor / adult_population * 100).round(1)

/usr/local/lib/python3.7/dist-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [50]:
# Calculate original child poverty rate
new_child_poor = (person.new_poor * person.child * person.weight).sum()
new_child_pov_rate = (new_child_poor / child_population * 100).round(1)

/usr/local/lib/python3.7/dist-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [89]:
inequality = pd.DataFrame(dict(
    metric=["Gini index", "Income share of top 10%", "Income share of top 1%"],
    current=[mdf.gini(person, "spm_resources", "weight"),
             mdf.top_10_pct_share(person, "spm_resources", "weight"),
             mdf.top_1_pct_share(person, "spm_resources", "weight")],
    reform=[mdf.gini(person, "spm_resources_new", "weight"),
            mdf.top_10_pct_share(person, "spm_resources_new", "weight"),
            mdf.top_1_pct_share(person, "spm_resources_new", "weight")],
))
inequality["chg"] = inequality.reform / inequality.current - 1
LABELS = dict(metric="Metric", chg="Change", current="Current", reform="Reform")
fig = px.bar(inequality.round(2), "metric", "chg", hover_data=["current", "reform"],
             labels=LABELS,
             title="Inequality impact of Ilhan Omar's Guaranteed Income Tax Credit")
fig.update_layout(yaxis_tickformat="%")
format_fig(fig)

In [88]:
def pov(p, resources):
    return mdf.poverty_rate(p, resources, "spm_povthreshold", "weight")

def deep_pov(p, resources):
    return mdf.deep_poverty_rate(p, resources, "spm_povthreshold", "weight")

poverty = pd.DataFrame(dict(
    age=["Child", "Adult", "All"] * 2,
    level=["Poverty"] * 3 + ["Deep poverty"] * 3,
    current=[pov(person[person.child], "spm_resources"),
             pov(person[person.adult], "spm_resources"),
             pov(person, "spm_resources"),
             deep_pov(person[person.child], "spm_resources"),
             deep_pov(person[person.adult], "spm_resources"),
             deep_pov(person, "spm_resources")],
    reform=[pov(person[person.child], "spm_resources_new"),
             pov(person[person.adult], "spm_resources_new"),
             pov(person, "spm_resources_new"),
             deep_pov(person[person.child], "spm_resources_new"),
             deep_pov(person[person.adult], "spm_resources_new"),
             deep_pov(person, "spm_resources_new")]
))
poverty["chg"] = poverty.reform / poverty.current - 1
poverty
fig = px.bar(poverty.round(3), "age", "chg", color="level", hover_data=["current", "reform"],
             labels=LABELS,
             title="Poverty impact of Ilhan Omar's Guaranteed Income Tax Credit",
             color_discrete_map={"Poverty": BLUE, "Deep poverty": GRAY})
fig.update_layout(yaxis_tickformat="%", barmode="group")
format_fig(fig)

In [57]:
# Poverty Chart
age=['All', 'Adult', 'Child']

y_og = [og_pov_rate, og_adult_pov_rate, og_child_pov_rate]
y_new = [new_pov_rate, new_adult_pov_rate, new_child_pov_rate]

fig = go.Figure(data=[
    go.Bar(name='Original', x=age, y=y_og,
           text=y_og, marker_color=BLUE),
    go.Bar(name='Omar Reform', x=age, y=y_new,
           text=y_new, marker_color=GRAY)
])


fig.update_layout(
                  font=dict(family='Roboto'),
                  plot_bgcolor='white',
                  title='Changes in poverty from Omar Gauranteed Income proposal'
                 )

fig.update_xaxes(
        tickangle = 0,
        tickfont = {"size": 14},
        title_standoff = 25)

fig.update_yaxes(
        title_text = "SPM poverty rate",
        ticksuffix ="%",
        tickfont = {'size':14},
        title_standoff = 25)

fig.update_traces(texttemplate='%{text}%')


# Change the bar mode
fig.update_layout(barmode='group')
format_fig(fig)

In [58]:
# No Kids
joint_x = [0, 150_000, 726_000]
joint_y = [28_800, 28_800, 0]

single_x = [0, 75_000, 363_000]
single_y = [14_400, 14_400, 0]

In [59]:
# Kids
# Two kids
joint_kids_x = [0, 150_000, 1_014_000]
joint_kids_y = [43_200, 43_200, 0]

head_kids_x = [0, 112_500, 688_500]
head_kids_y = [28_800, 28_800, 0]

In [60]:
fig = go.Figure()

fig.add_trace(go.Line(
    mode='lines',
    x=joint_x,
    y=joint_y,
    name='Married adults with no children',
    line_color=GRAY,
))


fig.add_trace(go.Line(
    mode='lines',
    x=single_x,
    y=single_y,
    name='Single adult with no children',
    line_color=BLUE,
))

fig.update_layout(
                  font=dict(family='Roboto'),
                  plot_bgcolor='white',
                  title='Income and Guaranteed Income transfer for childless homes in new Omar proposal'
                 )

fig.update_xaxes(
        tickangle = 0,
        title_text = "Income",
        tickfont = {"size": 14},
        tickprefix ="$",
        title_standoff = 25)

fig.update_yaxes(
        title_text = "Transfer",
        tickprefix ="$",
        tickfont = {'size':14},
        title_standoff = 25)

fig.update_layout(legend=dict(
    orientation="v",
    yanchor="bottom",
    y=0.8,
    xanchor="right",
    x=1
))

format_fig(fig)

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [61]:
fig = go.Figure()

fig.add_trace(go.Line(
    mode='lines',
    x=joint_kids_x,
    y=joint_kids_y,
    name='Married adults with two children',
    line_color=GRAY,
))


fig.add_trace(go.Line(
    mode='lines',
    x=head_kids_x,
    y=head_kids_y,
    name='Single adult with two children',
    line_color=BLUE,
))

fig.update_layout(
                  font=dict(family='Roboto'),
                  plot_bgcolor='white',
                  title='Income and Guaranteed Income transfer for two child homes in new Omar proposal'
                 )

fig.update_xaxes(
        tickangle = 0,
        title_text = "Income",
        tickfont = {"size": 14},
        tickprefix ="$",
        title_standoff = 25)

fig.update_yaxes(
        title_text = "Transfer",
        tickprefix ="$",
        tickfont = {'size':14},
        title_standoff = 25)

fig.update_layout(legend=dict(
    orientation="v",
    yanchor="bottom",
    y=0.8,
    xanchor="right",
    x=1
))

format_fig(fig)

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


